## 1- import data

In [2]:
import os
import json
import pandas as pd

# make sure folders exist
os.makedirs("data/analysis", exist_ok=True)

# load cleaned datasets
hr = pd.read_csv("data/cleaned/hr_clean.csv")
superstore = pd.read_csv("data/cleaned/superstore_clean.csv")
retail = pd.read_csv("data/cleaned/retail_clean.csv")

# check quick shapes
print("HR:", hr.shape, "Superstore:", superstore.shape, "Retail:", retail.shape)


HR: (1470, 36) Superstore: (9983, 21) Retail: (392692, 8)


## 2 - Convert Dates & Compute Revenue

In [6]:
# Convert dates
superstore['Order Date'] = pd.to_datetime(superstore['Order Date'], errors='coerce')
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'], errors='coerce')

# Revenue columns
superstore['Revenue'] = superstore['Sales']
retail['Revenue'] = retail['Quantity'] * retail['UnitPrice']


## 3- Total Revenue

In [8]:
total_revenue = superstore['Revenue'].sum() + retail['Revenue'].sum()
print("Total Revenue:", total_revenue)


Total Revenue: 11175480.3843


## 4 - Monthly Revenue Growth (MoM %)

In [15]:
monthly = superstore.groupby(superstore['Order Date'].dt.to_period("M"))['Revenue'].sum().reset_index()
monthly['Order Date'] = monthly['Order Date'].dt.to_timestamp()

rev_last = monthly['Revenue'].iloc[-1]
rev_prev = monthly['Revenue'].iloc[-2]
rev_mom_pct = (rev_last - rev_prev) / rev_prev * 100

print("Revenue Growth (MoM %):", rev_mom_pct)
monthly.to_csv("data/analysis/monthly_sales_for_kpis.csv", index=False)


Revenue Growth (MoM %): -29.399895017067646


## 5 - Average Order Value (AOV)

In [20]:
orders = superstore['Order ID'].nunique()  # make sure column is named 'Order ID'
aov = superstore['Revenue'].sum() / orders
print("Average Order Value:", aov)


Average Order Value: 457.3798701379173


## 6- Customer Churn Rate (Retail)

In [23]:
# Simple definition = % of customers who stopped buying after a cutoff date

cust_last_purchase = retail.groupby('CustomerID')['InvoiceDate'].max().reset_index()

# cutoff = 3 months before last date in dataset
cutoff = retail['InvoiceDate'].max() - pd.DateOffset(months=3)

churned = (cust_last_purchase['InvoiceDate'] < cutoff).sum()
total_customers = cust_last_purchase.shape[0]
churn_rate = churned / total_customers

print("Customer Churn Rate:", churn_rate)


Customer Churn Rate: 0.33310281235592437


## 7 - Employee Attrition Rate (HR)

In [26]:
attrition_rate = hr['AttritionFlag'].mean()  # since 1=Yes, 0=No
print("Employee Attrition Rate:", attrition_rate)


Employee Attrition Rate: 0.16122448979591836


## 8 - Save All KPIs

In [29]:
kpis = {
    "total_revenue": float(total_revenue),
    "revenue_mom_pct": float(rev_mom_pct),
    "average_order_value": float(aov),
    "customer_churn_rate": float(churn_rate),
    "employee_attrition_rate": float(attrition_rate)
}

# Save to JSON
with open("data/analysis/kpis.json", "w") as f:
    json.dump(kpis, f, indent=2)

print("✅ KPIs saved to data/analysis/kpis.json")
print(json.dumps(kpis, indent=2))


✅ KPIs saved to data/analysis/kpis.json
{
  "total_revenue": 11175480.3843,
  "revenue_mom_pct": -29.399895017067646,
  "average_order_value": 457.3798701379173,
  "customer_churn_rate": 0.33310281235592437,
  "employee_attrition_rate": 0.16122448979591836
}
